# LOB Execution Study - Interactive Mode

This notebook interactively calls the `run_execution_study.py` script functions to analyze market microstructure and execution strategies.

---

## Setup: Import the QuantLab Library

In [ ]:
# Add src to path and import required modules
import sys
from pathlib import Path

# Add the src directory to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Import quantlab modules
from quantlab.config import settings, setup_logging
from quantlab.microstructure import (
    LimitOrderBook, initialize_book,
    analyze_market_order, impact_by_size,
    order_flow_imbalance, calculate_ofi_from_book
)
from quantlab.execution import (
    VWAPExecutor, TWAPExecutor, compare_strategies
)

# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Setup
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
np.random.seed(42)

print("QuantLab Execution Engine loaded successfully!")
print(f"Project root: {project_root}")

---

## Step 1: Configure Execution Parameters

Set your order parameters interactively:

In [ ]:
# ============================================================
# INTERACTIVE CONFIGURATION
# Modify these parameters as needed
# ============================================================

# Order Parameters
ORDER_SIZE = 10000      # Number of shares to execute
ORDER_SIDE = 'BUY'      # 'BUY' or 'SELL'

# Market Parameters
SYMBOL = 'AAPL'
MID_PRICE = 150.00      # Initial mid price
SPREAD = 0.02           # Bid-ask spread (in dollars)
LEVELS = 20             # Number of price levels in the book
BASE_SIZE = 1000        # Base order size at each level

# Execution Parameters
VWAP_SLICES = 10        # Number of slices for VWAP execution

print(f"Configuration:")
print(f"  Symbol: {SYMBOL}")
print(f"  Order: {ORDER_SIDE} {ORDER_SIZE:,} shares")
print(f"  Initial Mid Price: ${MID_PRICE:.2f}")
print(f"  Spread: ${SPREAD:.2f}")
print(f"  Book Depth: {LEVELS} levels")
print(f"  VWAP Slices: {VWAP_SLICES}")

---

## Step 2: Initialize Limit Order Book (LOB)

In [ ]:
# Create a function to generate fresh order books
def create_book():
    """Factory function to create a fresh limit order book."""
    return initialize_book(
        symbol=SYMBOL,
        mid_price=MID_PRICE,
        spread=SPREAD,
        levels=LEVELS,
        base_size=BASE_SIZE
    )

# Create initial book
book = create_book()

print(f"Limit Order Book initialized for {SYMBOL}")
print(f"  Mid Price: ${book.get_mid_price():.2f}")
print(f"  Best Bid: ${book.get_best_bid()[0]:.2f}")
print(f"  Best Ask: ${book.get_best_ask()[0]:.2f}")
print(f"  Spread: ${book.get_spread():.4f}")

In [ ]:
# Visualize the order book
fig, ax = plt.subplots(figsize=(12, 6))

# Get bid and ask levels from snapshot
snapshot = book.get_snapshot(LEVELS)
bids = snapshot['bids']
asks = snapshot['asks']

bid_prices = [b['price'] for b in bids]
bid_sizes = [b['size'] for b in bids]
ask_prices = [a['price'] for a in asks]
ask_sizes = [a['size'] for a in asks]

# Plot bid side (green)
ax.barh(bid_prices, bid_sizes, height=SPREAD*0.8, color='green', alpha=0.7, label='Bids')

# Plot ask side (red)
ax.barh(ask_prices, [-s for s in ask_sizes], height=SPREAD*0.8, color='red', alpha=0.7, label='Asks')

# Add mid price line
mid_price = book.get_mid_price()
ax.axhline(y=mid_price, color='blue', linestyle='--', linewidth=2, label=f'Mid: ${mid_price:.2f}')

ax.set_xlabel('Order Size (shares)')
ax.set_ylabel('Price ($)')
ax.set_title(f'{SYMBOL} Limit Order Book', fontweight='bold', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nOrder Book Depth:")
print(f"  Total Bid Volume: {sum(bid_sizes):,} shares")
print(f"  Total Ask Volume: {sum(ask_sizes):,} shares")

---

## Step 3: Market Impact Analysis

Analyze how different order sizes impact the market:

In [ ]:
# Analyze market impact by order size
print("Analyzing market impact by order size...")
print("="*60)

sizes = [500, 1000, 2000, 3000, 5000, 7500, 10000, 15000, 20000]
impact_df = impact_by_size(create_book, sizes)

print(f"\nMarket Impact Analysis:")
print("-"*60)
print(f"{'Size':>10} {'Slippage (bps)':>15} {'Impact (bps)':>15} {'VWAP':>12}")
print("-"*60)

for _, row in impact_df.iterrows():
    print(f"{int(row['size']):>10,} {row['slippage_bps']:>15.1f} {row['impact_bps']:>15.1f} ${row['vwap']:>11.4f}")

In [ ]:
# Visualize market impact
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Slippage vs Size
ax1 = axes[0]
ax1.plot(impact_df['size'], impact_df['slippage_bps'], 'o-', color='blue', linewidth=2, markersize=8)
ax1.fill_between(impact_df['size'], 0, impact_df['slippage_bps'], alpha=0.2, color='blue')
ax1.set_xlabel('Order Size (shares)')
ax1.set_ylabel('Slippage (basis points)')
ax1.set_title('Slippage vs Order Size', fontweight='bold')
ax1.grid(True, alpha=0.3)

# Add annotation for current order size
current_slippage = impact_df[impact_df['size'] == ORDER_SIZE]['slippage_bps'].values
if len(current_slippage) > 0:
    ax1.axvline(x=ORDER_SIZE, color='red', linestyle='--', alpha=0.7)
    ax1.annotate(f'Your order: {ORDER_SIZE:,}\n{current_slippage[0]:.1f} bps', 
                 xy=(ORDER_SIZE, current_slippage[0]), 
                 xytext=(ORDER_SIZE*1.2, current_slippage[0]*1.3),
                 arrowprops=dict(arrowstyle='->', color='red'))

# Plot 2: VWAP vs Size
ax2 = axes[1]
ax2.plot(impact_df['size'], impact_df['vwap'], 'o-', color='green', linewidth=2, markersize=8)
ax2.axhline(y=MID_PRICE, color='black', linestyle='--', alpha=0.5, label=f'Mid Price: ${MID_PRICE:.2f}')
ax2.set_xlabel('Order Size (shares)')
ax2.set_ylabel('Execution VWAP ($)')
ax2.set_title('Execution VWAP vs Order Size', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(str(project_root / 'reports' / 'figures' / 'market_impact_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## Step 4: Execution Strategy Comparison

Compare different execution strategies:

In [ ]:
# Compare execution strategies
print(f"Comparing execution strategies for {ORDER_SIZE:,} {ORDER_SIDE} order...")
print("="*60)

comparison_df = compare_strategies(create_book, ORDER_SIZE, ORDER_SIDE, VWAP_SLICES)

print(f"\nExecution Strategy Comparison:")
print("-"*70)
print(f"{'Strategy':<15} {'Slippage (bps)':>15} {'VWAP':>12} {'Savings (bps)':>15}")
print("-"*70)

for _, row in comparison_df.iterrows():
    print(f"{row['strategy']:<15} {row['slippage_bps']:>15.1f} ${row['vwap']:>11.4f} {row['savings_vs_aggressive']:>15.1f}")

In [ ]:
# Visualize strategy comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Slippage comparison
ax1 = axes[0]
strategies = comparison_df['strategy'].values
slippages = comparison_df['slippage_bps'].values
colors = ['red' if s == 'Aggressive' else 'orange' if s == 'TWAP' else 'green' for s in strategies]

bars = ax1.bar(strategies, slippages, color=colors, alpha=0.7)
ax1.set_ylabel('Slippage (basis points)')
ax1.set_title('Execution Cost by Strategy', fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, slip in zip(bars, slippages):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{slip:.1f}', ha='center', va='bottom', fontweight='bold')

# Plot 2: Savings comparison
ax2 = axes[1]
savings = comparison_df['savings_vs_aggressive'].values
colors2 = ['gray' if s == 0 else 'green' for s in savings]

bars2 = ax2.bar(strategies, savings, color=colors2, alpha=0.7)
ax2.set_ylabel('Savings vs Aggressive (basis points)')
ax2.set_title('Cost Savings by Strategy', fontweight='bold')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, sav in zip(bars2, savings):
    if sav > 0:
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, 
                 f'+{sav:.1f}', ha='center', va='bottom', fontweight='bold', color='green')

plt.tight_layout()
plt.savefig(str(project_root / 'reports' / 'figures' / 'strategy_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## Step 5: Detailed VWAP Execution

In [ ]:
# Run detailed VWAP execution
print(f"Running VWAP execution ({VWAP_SLICES} slices)...")
print("="*60)

book = create_book()
vwap_exec = VWAPExecutor(book)
vwap_result = vwap_exec.execute(ORDER_SIZE, ORDER_SIDE, VWAP_SLICES)

print(f"\nVWAP Execution Detail:")
print("-"*40)
print(f"{'Slice':>6} {'Size':>10} {'VWAP':>12}")
print("-"*40)

slice_size = ORDER_SIZE // VWAP_SLICES
for i, vwap in enumerate(vwap_result.slice_vwaps):
    print(f"{i+1:>6} {slice_size:>10,} ${vwap:>11.4f}")

print("-"*40)
print(f"{'Total':>6} {ORDER_SIZE:>10,} ${vwap_result.vwap:>11.4f}")

In [ ]:
# Visualize VWAP execution trajectory
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: VWAP by slice
ax1 = axes[0]
slices = range(1, len(vwap_result.slice_vwaps) + 1)
ax1.bar(slices, vwap_result.slice_vwaps, color='steelblue', alpha=0.7)
ax1.axhline(y=MID_PRICE, color='green', linestyle='--', linewidth=2, label=f'Initial Mid: ${MID_PRICE:.2f}')
ax1.axhline(y=vwap_result.vwap, color='red', linestyle='-', linewidth=2, label=f'Avg VWAP: ${vwap_result.vwap:.4f}')
ax1.set_xlabel('Slice Number')
ax1.set_ylabel('VWAP ($)')
ax1.set_title('VWAP by Execution Slice', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Cumulative execution
ax2 = axes[1]
cumulative_shares = [slice_size * (i+1) for i in range(VWAP_SLICES)]
cumulative_cost = [sum(vwap_result.slice_vwaps[:i+1]) * slice_size / (slice_size * (i+1)) for i in range(VWAP_SLICES)]

ax2.plot(cumulative_shares, cumulative_cost, 'o-', color='blue', linewidth=2, markersize=8)
ax2.axhline(y=MID_PRICE, color='green', linestyle='--', alpha=0.5, label=f'Mid: ${MID_PRICE:.2f}')
ax2.fill_between(cumulative_shares, MID_PRICE, cumulative_cost, alpha=0.2, color='red')
ax2.set_xlabel('Cumulative Shares Executed')
ax2.set_ylabel('Average Execution Price ($)')
ax2.set_title('Cumulative Execution Profile', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(str(project_root / 'reports' / 'figures' / 'vwap_execution_detail.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## Step 6: Execution Summary

In [ ]:
# Generate execution summary
aggressive = comparison_df[comparison_df['strategy'] == 'Aggressive'].iloc[0]
vwap_row = comparison_df[comparison_df['strategy'] == 'VWAP'].iloc[0]

savings_bps = aggressive['slippage_bps'] - vwap_row['slippage_bps']
savings_dollar = savings_bps / 10000 * ORDER_SIZE * MID_PRICE

print("\n" + "="*70)
print("                    EXECUTION STUDY SUMMARY")
print("="*70)
print(f"\nOrder Parameters:")
print(f"  Symbol: {SYMBOL}")
print(f"  Side: {ORDER_SIDE}")
print(f"  Size: {ORDER_SIZE:,} shares")
print(f"  Initial Mid Price: ${MID_PRICE:.2f}")
print(f"  Notional Value: ${ORDER_SIZE * MID_PRICE:,.0f}")

print("\n" + "-"*70)
print("EXECUTION RESULTS")
print("-"*70)
print(f"  Aggressive Execution:")
print(f"    - VWAP: ${aggressive['vwap']:.4f}")
print(f"    - Slippage: {aggressive['slippage_bps']:.1f} bps (${aggressive['slippage_bps']/10000 * ORDER_SIZE * MID_PRICE:,.2f})")

print(f"\n  VWAP Execution ({VWAP_SLICES} slices):")
print(f"    - VWAP: ${vwap_row['vwap']:.4f}")
print(f"    - Slippage: {vwap_row['slippage_bps']:.1f} bps (${vwap_row['slippage_bps']/10000 * ORDER_SIZE * MID_PRICE:,.2f})")

print("\n" + "-"*70)
print("COST SAVINGS ANALYSIS")
print("-"*70)
print(f"  Slippage Reduction: {savings_bps:.1f} basis points")
print(f"  Dollar Savings: ${savings_dollar:,.2f}")
print(f"  Cost Reduction: {savings_bps/aggressive['slippage_bps']*100:.1f}% vs aggressive execution")

print("\n" + "="*70)

In [ ]:
# Save results
settings.ensure_dirs()
output_path = settings.processed_data_dir / "execution_results_interactive.csv"

comparison_df.to_csv(output_path, index=False)

impact_output = settings.processed_data_dir / "impact_analysis_interactive.csv"
impact_df.to_csv(impact_output, index=False)

print(f"Results saved to:")
print(f"  - {output_path}")
print(f"  - {impact_output}")

print("\nExecution Comparison:")
display(comparison_df)

print("\nImpact Analysis:")
display(impact_df)

---

## Step 7: Interactive Experimentation

Try different scenarios:

In [ ]:
# Experiment: Compare different VWAP slice counts
print("Comparing different VWAP slice configurations...")
print("="*60)

slice_counts = [5, 10, 15, 20, 25]
slice_results = []

for n_slices in slice_counts:
    book = create_book()
    vwap_exec = VWAPExecutor(book)
    result = vwap_exec.execute(ORDER_SIZE, ORDER_SIDE, n_slices)
    slippage = (result.vwap - MID_PRICE) / MID_PRICE * 10000
    slice_results.append({
        'slices': n_slices,
        'vwap': result.vwap,
        'slippage_bps': slippage
    })

slice_comparison = pd.DataFrame(slice_results)
display(slice_comparison)

# Visualize
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(slice_comparison['slices'], slice_comparison['slippage_bps'], 'o-', color='blue', linewidth=2, markersize=10)
ax.set_xlabel('Number of VWAP Slices')
ax.set_ylabel('Slippage (basis points)')
ax.set_title('Impact of VWAP Slice Count on Execution Cost', fontweight='bold')
ax.grid(True, alpha=0.3)

# Find optimal
optimal = slice_comparison.loc[slice_comparison['slippage_bps'].idxmin()]
ax.scatter([optimal['slices']], [optimal['slippage_bps']], color='green', s=200, zorder=5, label=f"Optimal: {int(optimal['slices'])} slices")
ax.legend()

plt.tight_layout()
plt.show()

print(f"\nOptimal configuration: {int(optimal['slices'])} slices with {optimal['slippage_bps']:.1f} bps slippage")

---

## Notebook Complete

You can now:
1. Modify order size, side, and other parameters in Step 1
2. Change market microstructure parameters (spread, depth, etc.)
3. Experiment with different VWAP configurations
4. Compare execution costs across scenarios